#  Final Project: Etsy Marketplace Exploration

## Data Aquisition

In [29]:
# import HTML tools
from lxml import html
import xml.etree.ElementTree as ET

In [ ]:
# read in the list of etsy page categories
import pandas as pd
etsy_pages = pd.read_csv("etsy_category_pages.csv")


In [ ]:
page_cat = etsy_pages['page'].tolist()

In [ ]:
# generate list of urls for each category

urls = []
start = "https://www.etsy.com/c/"

for x in page_cat:
    urls.append(start + x)

In [ ]:
# All of these categories get a successful response 
# Now that we have a list of catgories with associated web pages, we can scrape subcategories
# After we collect subcategores we can get the shop names from each subcategory page

In [ ]:
urls

In [ ]:
# Find all html elements which contain the links to the subcategory pages
import requests
import lxml
sub_elements = []

for url in urls:
    #send the request
    content = requests.get(url)
    #save the page source code to a string called content_string
    content_string = content.text.encode("utf-8")
    #pass the page source to our html parse
    doc = lxml.html.document_fromstring(content_string)
    # Find html element containing each subcategory
    element = doc.xpath("//li[@class='pb-xs-1 pl-xs-0']")
    sub_elements.append(element)

In [ ]:
# Get urls for each of the subcategories and store them in a list
sub_links = []

for element in sub_elements:
    for x in element:
        child = x.getchildren()
        new = child[0].attrib['href']
        sub_links.append(new)

In [ ]:
# Save subcategory links in a csv file
import csv

with open('subcategory_links.csv', 'w') as csvfile:
    fieldnames=['link']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    # iterate through category and subcategory
    for item in sub_links:
        writer.writerow({'link': item})

In [ ]:
# open csv subcategory link file
# create new pandas dataframe called catdf

newdata = pd.read_csv("subcategory_links.csv",sep="/",names=('0','1','2','3','top_category','link_address'))
del(newdata['0'],newdata['1'],newdata['2'],newdata['3'])
catdf = newdata.dropna()


In [ ]:
# extract subcatgory name from link and save as new csv file

def clean_links(text):
    return text.split("?")[0]

catdf['sub_category'] = catdf['link_address'].apply(clean_links)


In [ ]:
catdf.head()

In [ ]:
# Function to get urls for pages 2-250 in each subcategory
# https://www.etsy.com/c/jewelry/bracelets?page=24

def get_page2(row):
    urls = []
    start = "https://www.etsy.com/c/"
    for i in range(2,250):
        urls.append(start + row['top_category'] + "/" + row['sub_category'] + "?page=" + str(i))
    return ",".join(urls)

In [ ]:
# Apply function and story multiple page urls in column called page_nums

catdf['page_nums'] = catdf.apply(get_page2, axis=1)

In [ ]:
catdf.head()

In [ ]:
sub_links = pd.read_csv("subcategory_links.csv")

In [ ]:
# join dataframes

catdf = catdf.reset_index()
catdf = catdf.join(sub_links)
del catdf['index']
del catdf['link_address']

catdf.head(5)

In [ ]:
# finds all shop names from html source string and appends to list
# returns string
import time

def get_shop_names(some_link):
    time.sleep(2)
    content = some_link
    source_code = requests.get(content)
    source_string = source_code.text.encode("utf-8")
    j = []
    doc = lxml.html.document_fromstring(source_string)
    tags = doc.xpath("//div[@class='card-meta-row-item text-truncate overflow-hidden card-shop-name']")
    for tag in tags:
        j.append(tag.text)
    return ",".join(j)

In [ ]:
new_column = sub_links['link'].apply(get_shop_names)

In [ ]:
catdf['raw_shops'] = new_column

In [4]:
# turn string into list of strings

def link_list(a_string):
    foo = a_string.split(",")
    return foo

In [ ]:
catdf.head()

In [ ]:
catdf['more_shops'] = ""

In [ ]:
catdf['even_more_shops'] = ""

In [ ]:
# run get_shop_names on page_nums
# j is length of catdf
# i is page of results you want to see (up to 250)

for j in range(0,161):
    trial = link_list(catdf['page_nums'][j])
    k = []
    for i in range(20,25):
        one = get_shop_names(trial[i])
        k.append(one)
    catdf['even_more_shops'][j] = ",".join(k)

In [ ]:
catdf

In [ ]:
catdf.to_csv(path_or_buf="shop_names")

In [ ]:
# takes raw string containing shop names
#returns cleaned with shop name only separated by comas

import re

def clean_name(messy_name):
    new_list = []
    for name in messy_name:
        finder = re.compile("\n" + "(.*?)" + "\n", re.IGNORECASE)
        matches = finder.findall(name)
        no_replace = re.compile("\s+")
        p = no_replace.sub("", matches[0])
        new_list.append(p)
    return ",".join(new_list)

In [ ]:
catdf['all_shops'] = ""

In [ ]:
catdf.tail(83)

In [ ]:
catdf['even_more_shops'][99]

In [ ]:
s = catdf['more_shops'][160]

In [ ]:
s

In [ ]:
catdf['even_more_shops'][160] = s

In [ ]:
# clean more_shops and store names in all_shops
# run 0-161

for i in range(0,161):
    bar = link_list(catdf['more_shops'][i])
    catdf['all_shops'][i] = clean_name(bar)

In [ ]:
catdf['all2_shops'] = ""

In [ ]:
catdf['all3_shops'] = ""

In [ ]:
# clean even_more_shops and store names in all_shops
# run 0-161

for i in range(0,161):
    bar = link_list(catdf['even_more_shops'][i])
    catdf['all2_shops'][i] = clean_name(bar)

In [ ]:
# clean raw_shops and store names in all3_shops
# run 0-161

for i in range(0,161):
    bar = link_list(catdf['raw_shops'][i])
    catdf['all3_shops'][i] = clean_name(bar)

In [ ]:
catdf.tail(3)

In [ ]:
def appendcols(row):
    s = row['all_shops'] + row['all_shops'] + row['all_shops']
    return s

In [ ]:
catdf['shop_names'] = ""

In [ ]:
catdf['shop_names'] = catdf.apply(appendcols, axis=1)

In [ ]:
catdf['all_shops'][2] == catdf['all2_shops'][2]

In [ ]:
final_catdf = catdf[['top_category','sub_category','shop_names']]
#df2 = df[["col1", "col2", "col3"]]

In [ ]:
s = final_catdf['shop_names'][121]

In [ ]:
ss = s.encode("utf-8")

In [ ]:
type(ss)

In [ ]:
final_catdf['shop_names'][121] = ss

In [ ]:
for i in range(120,161):
    print i
    print type(final_catdf['shop_names'][i])

In [ ]:
final_catdf.to_csv(path_or_buf="shop_names")

# Check Point

In [1]:
import pandas as pd

final_catdf = pd.read_csv("shop_names")

In [2]:
final_catdf

,Unnamed: 0,top_category,sub_category,shop_names
0,0,accessories,hair-accessories,"TheIvoryWillowCo,ChicBlue,TheBridesStore,Lovel..."
1,1,accessories,hats-and-caps,"RockyMountainHatCo,RambleBabyBoutique,Rawsells..."
2,2,accessories,baby-accessories,"SkylarnMe,SweetAndStitched,LacoHandMade,Simply..."
3,3,accessories,scarves-and-wraps,"MiracleShine,ScarfClub,Zojanka,LTCoutureAtelie..."
4,4,accessories,keychains-and-lanyards,"artbymorgie,accessoriesbyjamiep,NiOuiNinoucheG..."
5,5,accessories,suit-and-tie-accessories,"BridalCloset,LoveVanillaDew,ElegantCuff,TieObs..."
6,6,accessories,patches-and-pins,"dannybrito,ilikeCATSshop,DistinctDesignsUnltd,..."
7,7,accessories,belts-and-suspenders,"RhythmicMetal,ISHAOR,armoniia,TheLeatherBoy,Th..."
8,8,accessories,costume-accessories,"Merimask,superheroforparty,Customownnecklace,K..."
9,9,accessories,sunglasses-and-eyewear,"SunglassMuseum,retroandme,Propwood,CarvedAcces..."


In [6]:
every_shop = []

for i in range(len(final_catdf['shop_names'])):
    something = link_list(final_catdf['shop_names'][i])
    for x in something:
        every_shop.append(x)

In [7]:
shop_df = pd.DataFrame(data = every_shop)

In [9]:
shop_df['shop_name'] = shop_df[0]

In [10]:
shop_df

,0,shop_name
0,TheIvoryWillowCo,TheIvoryWillowCo
1,ChicBlue,ChicBlue
2,TheBridesStore,TheBridesStore
3,LovelyLittlesandCo,LovelyLittlesandCo
4,masonandharlow,masonandharlow
5,RoyalSistersBoutique,RoyalSistersBoutique
6,LeftbankSV,LeftbankSV
7,avtboutique,avtboutique
8,thomaspark,thomaspark
9,buttercupsbows,buttercupsbows


In [11]:
new = shop_df.drop_duplicates(inplace = True)

In [12]:
shop_df['review_links'] = ""

In [13]:
shop_df = shop_df.reset_index()

In [19]:
len(shop_df)

13211

In [22]:
# generate urls for each shop name in the list

for i in range(10000,13211):
    x = shop_df['shop_name'][i]
    url = ("https://www.etsy.com/shop/"+ x +"/reviews")
    shop_df['review_links'][i] = url
    print i

/Users/jill/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


10000
10001
10002
10003
10004
10005
10006
10007
10008
10009
10010
10011
10012
10013
10014
10015
10016
10017
10018
10019
10020
10021
10022
10023
10024
10025
10026
10027
10028
10029
10030
10031
10032
10033
10034
10035
10036
10037
10038
10039
10040
10041
10042
10043
10044
10045
10046
10047
10048
10049
10050
10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
1016

In [23]:
shop_df

,index,0,shop_name,review_links
0,0,TheIvoryWillowCo,TheIvoryWillowCo,https://www.etsy.com/shop/TheIvoryWillowCo/rev...
1,1,ChicBlue,ChicBlue,https://www.etsy.com/shop/ChicBlue/reviews
2,2,TheBridesStore,TheBridesStore,https://www.etsy.com/shop/TheBridesStore/reviews
3,3,LovelyLittlesandCo,LovelyLittlesandCo,https://www.etsy.com/shop/LovelyLittlesandCo/r...
4,4,masonandharlow,masonandharlow,https://www.etsy.com/shop/masonandharlow/reviews
5,5,RoyalSistersBoutique,RoyalSistersBoutique,https://www.etsy.com/shop/RoyalSistersBoutique...
6,6,LeftbankSV,LeftbankSV,https://www.etsy.com/shop/LeftbankSV/reviews
7,7,avtboutique,avtboutique,https://www.etsy.com/shop/avtboutique/reviews
8,8,thomaspark,thomaspark,https://www.etsy.com/shop/thomaspark/reviews
9,9,buttercupsbows,buttercupsbows,https://www.etsy.com/shop/buttercupsbows/reviews


In [63]:
shop_df['review_dates'] = ""

In [64]:
shop_df['ratings'] = ""

In [65]:
shop_df['item_details'] = ""

In [ ]:
# get dates, ratings and details for each shop

In [107]:
for i in range(4000,6000):
        
    # send request to url
    time.sleep(.5)
    content = shop_df['review_links'][i]
    source_code = requests.get(content)
    source_string = source_code.text.encode("utf-8")
    doc = lxml.html.document_fromstring(source_string)
    
    # find xpath with review dates
    p_elements = []
    a_elements = []
    list_dates = []
    tags = doc.xpath("//div[@class='mt-xs-2 mb-xs-2']")
    for tag in tags:
        p_elements.append(tag.getchildren())
    for p in p_elements:
        for x in p:
            a_elements.append(x.getchildren())
    for element in a_elements:
        for x in element:
            list_dates.append(x.tail)
    shop_df['review_dates'][i] = ",".join(list_dates)

    # find xpath with review ratings
    span_elements = []
    tags2 = doc.xpath("//span[@class='screen-reader-only']") 
    for tag in tags2:
        span_elements.append(tag.text)
    shop_df['ratings'][i] = ",".join(span_elements)
    
    # find xpath with item details
    n_elements = []
    m_elements = []
    list_details = []
    tags3 = doc.xpath("//div[@class='flag-body hide-xs hide-sm']")
    for tag in tags3:
        n_elements.append(tag.getchildren())
    for n in n_elements:
        for x in n:
            m_elements.append(x.getchildren())
    for m in m_elements:
        for x in m:
            list_details.append(x.text)
    shop_df['item_details'][i] = ",".join(list_details)
    
    print i

/Users/jill/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jill/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jill/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


4000
4001
4002
4003
4004
4005
4006
4007
4008
4009
4010
4011
4012
4013
4014
4015
4016
4017
4018
4019
4020
4021
4022
4023
4024
4025
4026
4027
4028
4029
4030
4031
4032
4033
4034
4035
4036
4037
4038
4039
4040
4041
4042
4043
4044
4045
4046
4047
4048
4049
4050
4051
4052
4053
4054
4055
4056
4057
4058
4059
4060
4061
4062
4063
4064
4065
4066
4067
4068
4069
4070
4071
4072
4073
4074
4075
4076
4077
4078
4079
4080
4081
4082
4083
4084
4085
4086
4087
4088
4089
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4106
4107
4108
4109
4110
4111
4112
4113
4114
4115
4116
4117
4118
4119
4120
4121
4122
4123
4124
4125
4126
4127
4128
4129
4130
4131
4132
4133
4134
4135
4136
4137
4138
4139
4140
4141
4142
4143
4144
4145
4146
4147
4148
4149
4150
4151
4152
4153
4154
4155
4156
4157
4158
4159
4160
4161
4162
4163
4164
4165
4166
4167
4168
4169
4170
4171
4172
4173
4174
4175
4176
4177
4178
4179
4180
4181
4182
4183
4184
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199


In [80]:
shop_df[2000:4000]

,index,0,shop_name,review_links,review_dates,ratings,item_details
2000,8338,daphnenen,daphnenen,https://www.etsy.com/shop/daphnenen/reviews,"on Aug 17, 2016\n , on Aug...","Search,5 out of 5 stars,5 out of 5 stars,5 out...","womens backpack crossbody bag, eco friendly ba..."
2001,8341,BennyBeeLeather,BennyBeeLeather,https://www.etsy.com/shop/BennyBeeLeather/reviews,"on Aug 16, 2016\n , on Aug...","Search,5 out of 5 stars,5 out of 5 stars,5 out...","Leather Messenger Bag, Leather Briefcase 15"", ..."
2002,8342,Enchantingcrea,Enchantingcrea,https://www.etsy.com/shop/Enchantingcrea/reviews,"on Aug 18, 2016\n , on Aug...","Search,5 out of 5 stars,5 out of 5 stars,5 out...","Harris Tweed purse, coin purse, change purse, ..."
2003,8343,ARoadThroughTime,ARoadThroughTime,https://www.etsy.com/shop/ARoadThroughTime/rev...,"on Aug 18, 2016\n , on Aug...","Search,4.5 out of 5 stars,4.5 out of 5 stars,5...","Vintage Military Bag 1980's, Green Cotton Canv..."
2004,8353,NickysLeather,NickysLeather,https://www.etsy.com/shop/NickysLeather/reviews,"on Aug 21, 2016\n , on Aug...","Search,5 out of 5 stars,5 out of 5 stars,5 out...",GLADIATOR SANDALS from Full Grain Leather Hand...
2005,8354,BiciByBar,BiciByBar,https://www.etsy.com/shop/BiciByBar/reviews,"on Aug 2, 2016\n , on Apr ...","Search,5 out of 5 stars,5 out of 5 stars,5 out...",Bicycle pannier/Navy blue canvas and leather p...
2006,8356,RobinsNestNton,RobinsNestNton,https://www.etsy.com/shop/RobinsNestNton/reviews,"on Aug 3, 2016\n , on Aug ...","Search,5 out of 5 stars,5 out of 5 stars,5 out...",A Vintage Leather Masonic Pouch - Masonic Rega...
2007,8357,SassySparrowBoutique,SassySparrowBoutique,https://www.etsy.com/shop/SassySparrowBoutique...,"on Aug 19, 2016\n , on Aug...","Search,5 out of 5 stars,5 out of 5 stars,5 out...","Womens Tote Bag, Large Tote Bag, Leather Tote ..."
2008,8360,Filzkraft,Filzkraft,https://www.etsy.com/shop/Filzkraft/reviews,"on Aug 17, 2016\n , on Aug...","Search,5 out of 5 stars,5 out of 5 stars,5 out...",Felt Passport Holder Passport Cover Card Holde...
2009,8361,DeReborn,DeReborn,https://www.etsy.com/shop/DeReborn/reviews,"on Jul 31, 2016\n , on Jul...","Search,4.5 out of 5 stars,4.5 out of 5 stars,5...","Leather Pouch Clutch Wallet with Tassel Edge,J..."


In [105]:
string = 0
unicodechar = 0
fix_unicode = []

In [106]:
# check if string has any unicode characters

for i in range(0,4000):
    if type(shop_df['item_details'][i]) == str:
        string = string + 1
    if type(shop_df['item_details'][i]) != str:
        #s = shop_df['review_dates'][i]
        #ss = s.encode("utf-8")
        fix_unicode.append(i)
        #shop_df['review_dates'][i] = shop_df['review_dates'][i].encode("utf-8")
        unicodechar = unicodechar + 1

2
23
47
61
92
99
128
202
225
250
251
266
302
318
337
389
444
453
491
505
548
568
576
601
661
673
687
696
700
722
729
737
752
823
844
848
854
859
874
878
897
900
904
939
949
962
972
978
981
998
1005
1038
1040
1044
1053
1071
1074
1075
1094
1095
1113
1128
1132
1200
1229
1232
1244
1278
1298
1318
1361
1369
1389
1395
1413
1424
1436
1441
1449
1461
1464
1492
1506
1511
1524
1554
1569
1575
1631
1637
1662
1676
1685
1699
1725
1726
1743
1767
1780
1799
1836
1867
1874
1907
1919
1921
1960
2004
2058
2066
2092
2142
2153
2155
2165
2179
2180
2182
2190
2193
2201
2214
2224
2235
2236
2247
2262
2272
2275
2281
2285
2311
2323
2328
2330
2333
2345
2353
2355
2369
2375
2376
2394
2395
2437
2453
2462
2466
2490
2498
2542
2559
2591
2619
2662
2692
2694
2698
2702
2721
2739
2750
2772
2775
2778
2781
2794
2796
2813
2824
2832
2834
2843
2856
2902
2947
2954
2961
2991
3007
3058
3077
3114
3140
3148
3156
3167
3171
3174
3207
3220
3228
3259
3281
3282
3303
3349
3357
3360
3386
3410
3416
3474
3534
3558
3585
3599
3605
3640
3642
3659
36

In [104]:
unicodechar

229

In [ ]:
for each in fix_unicode:
    s = shop_df['item_details'][each]
    ss = s.encode("utf-8")
    shop_df['item_details'][each] = ss
    

In [78]:
# write to csv file

shop_df.to_csv(path_or_buf = "review_data_page1")

UnicodeEncodeError: 'ascii' codec can't encode character u'\xe9' in position 114: ordinal not in range(128)

# Check Point

In [ ]:
# create a new df equal to shopdf['shop_name']
# this new df will store the review_links for pages 2 and up of reviews

review_df = shop_df['shop_name']

In [ ]:
# Function to get urls for review pages 2-50 for each shop
# https://www.etsy.com/shop/MamaBleuDesigns/reviews?ref=pagination&page=3

def get_review2(row):
    urls = []
    start = "https://www.etsy.com/shop/"
    for i in range(2,50):
        urls.append(start + row['shop_name'] + "/reviews?ref=pagination&page=" + str(i))
    return ",".join(urls)

In [ ]:
# Apply function and story multiple page urls in column called page_nums

review_df['page_nums'] = review_df.apply(get_review2, axis=1)

In [ ]:
review_df['review_dates'] = ""

In [ ]:
review_df['ratings'] = ""

In [ ]:
review_df['item_details'] = ""

In [ ]:
for i in range(0,2):
    try:    
        # send request to url
        time.sleep(1)
        content = review_df['page_nums'][i]
        source_code = requests.get(content)
        source_string = source_code.text.encode("utf-8")
        doc = lxml.html.document_fromstring(source_string)
    
        # find xpath with review dates
        p_elements = []
        a_elements = []
        list_dates = []
        tags = doc.xpath("//div[@class='mt-xs-2 mb-xs-2']")
        for tag in tags:
            p_elements.append(tag.getchildren())
        for p in p_elements:
            for x in p:
                a_elements.append(x.getchildren())
        for element in a_elements:
            for x in element:
                list_dates.append(x.tail)
        shop_df['review_dates'][i] = ",".join(list_dates)

        # find xpath with review ratings
        span_elements = []
        tags2 = doc.xpath("//span[@class='screen-reader-only']") 
        for tag in tags2:
            span_elements.append(tag.text)
        shop_df['ratings'][i] = ",".join(span_elements)
    
        # find xpath with item details
        n_elements = []
        m_elements = []
        list_details = []
        tags3 = doc.xpath("//div[@class='flag-body hide-xs hide-sm']")
        for tag in tags3:
            n_elements.append(tag.getchildren())
        for n in n_elements:
            for x in n:
                m_elements.append(x.getchildren())
        for m in m_elements:
            for x in m:
                list_details.append(x.text)
        shop_df['item_details'][i] = ",".join(list_details)
    
        print i

In [ ]:
shopdf.to_csv(path_or_buf = "review_data_page2")